In [ ]:
import Pkg
Pkg.activate(".")

# not for the faint of heart!
# Pkg.update()

pkgs = [
"ArgParse",
"Base64",
"BioSequences",
"DataFrames",
"Dates",
"DelimitedFiles",
"FASTX",
"GLM",
"HTTP",
"JSON",
"Graphs",
"MetaGraphs",
"MD5",
"Statistics",
"StatsPlots",
"uCSV",
"CodecZlib",
"YAML",
"Revise",
"Kmers",
"StatsBase",
"ProgressMeter"
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

import Mycelia

In [ ]:
data_dir = joinpath(dirname(pwd()), "data")

In [ ]:
# create full refseq database with taxonomic IDs from scratch

# blastdb_dir = mkpath("$(homedir())/workspace/blastdbs/refseq")

# refseq_metadata = Mycelia.load_refseq_metadata();

# gcf_to_url_map = Dict(row["# assembly_accession"] => Mycelia.ncbi_ftp_path_to_url(ftp_path = row["ftp_path"], extension = "genomic.fna.gz") for row in DataFrames.eachrow(refseq_metadata))

# expected_list_of_fasta_files = basename.(values(gcf_to_url_map))
# actual_list_of_fasta_files = filter(x -> occursin(r"genomic\.fna\.gz$", x), readdir(blastdb_dir))
# fasta_files_to_remove = setdiff(actual_list_of_fasta_files, expected_list_of_fasta_files)

# for f in fasta_files_to_remove
#     rm(joinpath(blastdb_dir, f))
# end

# missing_fasta_files = setdiff(expected_list_of_fasta_files, actual_list_of_fasta_files)

# # using ProgressMeter
# # p = Progress(10)
# # Threads.@threads for i in 1:10
# #     sleep(2*rand())
# #     next!(p)
# # end
# # finish!(p)

# ProgressMeter.@showprogress for f in missing_fasta_files
#     GCF_identifier = join(split(f, '_')[1:2], '_')
#     url = gcf_to_url_map[GCF_identifier]
#     out_f = joinpath(blastdb_dir, f)
#     # @show f, out_f
#     if !isfile(out_f)
#         download(url, out_f)
#     end
# end

# actual_list_of_fasta_files = filter(x -> occursin(r"genomic\.fna\.gz$", x), readdir(blastdb_dir))
# # @assert actual_list_of_fasta_files == expected_list_of_fasta_files
# list_of_fasta_files = intersect(actual_list_of_fasta_files, expected_list_of_fasta_files)

# ProgressMeter.@showprogress for f in list_of_fasta_files
#     f_path = joinpath(blastdb_dir, f)
#     headers_file = f_path * ".headers.txt"
#     if !isfile(headers_file)
#         p1 = pipeline(`gzip -dc $(f_path)`, `grep "^>"`)
#         p2 = pipeline(p1, headers_file)
#         run(p2)
#     end
# end

# # find -name "*_genomic.fna.gz" > genome_list.txt
# # head -n3 genome_list.txt > genome_list.3.txt

# # $ parallel --shellquote
# # parallel: Warning: Input is read from the terminal. You either know what you
# # parallel: Warning: are doing (in which case: YOU ARE AWESOME!) or you forgot
# # parallel: Warning: ::: or :::: or to pipe data into parallel. If so
# # parallel: Warning: consider going through the tutorial: man parallel_tutorial
# # parallel: Warning: Press CTRL-D to exit.
# # gzip -dc {} | grep "^>" > {}.headers.txt
# # parallel gzip\ -dc\ \{\}\ \|\ grep\ \"\^\>\"\ \>\ \{\}.headers.txt :::: genome_list.3.txt

# # parallel gzip\ -dc\ \{\}\ \|\ grep\ \"\^\>\"\ \>\ \{\}.headers.txt :::: genome_list.txt

# # NOTE instead of making a giant joint fasta, it seems possible to make a db for each fasta, and then link them
# # this would be more efficient to update and revise on the fly
# # would also probably be more storage efficient
# # will try both

# # blastdb_aliastool -dblist "database_1 database_2 and so on" -dbtype nucl -out your_linked_db_name -title "My full DB"
# # Late to the conversation but I realized something important.
# # The above may not work if your concatenation results in a fasta file larger than "1000000000" bytes. The makeblastdb output specifies this number.
# # The safer option is to build a database from each fasta file (after verifying that the file is less than 1000000000B and splitting if necessary; latter can be done like this: https://biopython.org/wiki/Split_large_file).
# # Then you can use the blastdb_aliastool like so (adapted from NCBI's manual):
# # blastdb_aliastool -dblist "database_1 database_2 and so on" -dbtype nucl -out your_linked_db_name -title "My full DB"

# # NOTE, CAN PROBABLY SKIP MAKING A JOINT FILE
# joint_fasta_file = joinpath(blastdb_dir, "refseq.fna.gz")
# if !isfile(joint_fasta_file)
#     # 39311.386470 seconds
#     # 10 hours!!
#     @time Mycelia.merge_fasta_files(joinpath.(blastdb_dir, list_of_fasta_files), joint_fasta_file)
# else
#     @info "$(joint_fasta_file) present, remove to regenerate..."
# end

# fasta_to_taxon_id_map = gcf_to_url_map = Dict(basename(Mycelia.ncbi_ftp_path_to_url(ftp_path = row["ftp_path"], extension = "genomic.fna.gz")) => row["taxid"] for row in DataFrames.eachrow(refseq_metadata))

#  # -taxid_map <File_In>
#  #   Text file mapping sequence IDs to taxonomy IDs.
#  #   Format:<SequenceId> <TaxonomyId><newline>
#  #    * Requires:  parse_seqids
#  #    * Incompatible with:  taxid

# # Progress: 100%|█████████████████████████████████████████| Time: 1:16:56
# taxon_map_file = joint_fasta_file * ".taxon_map.txt"
# if !isfile(taxon_map_file)
#     open(taxon_map_file, "w") do io
#         ProgressMeter.@showprogress for f in list_of_fasta_files
#             f_path = joinpath(blastdb_dir, f)
#             headers_file = f_path * ".headers.txt"
#             taxon_id = fasta_to_taxon_id_map[f]
#             for line in eachline(headers_file)
#                 sequence_id = replace(first(split(line)), ">" => "")
#                 println(io, "$(sequence_id) $(taxon_id)")
#             end
#         end
#     end
# else
#     @info "$(taxon_map_file) already present..."
# end

# # consider trying these to improve performance later???
# # @time run(`dustmasker -in $(decompressed_joint_fasta_file) -infmt fasta -parse_seqids -outfmt maskinfo_asn1_bin -out $(decompressed_joint_fasta_file).asnb`)
# # dustmasker -in hs_chr -infmt blastdb -parse_seqids -outfmt maskinfo_asn1_bin -out hs_chr_dust.asnb
# # makeblastdb -in hs_chr –input_type blastdb -dbtype nucl -parse_seqids -mask_data hs_chr_mask.asnb -out hs_chr -title "Human Chromosome, Ref B37.1"

# db_title = "Refseq $(Dates.today())"

# # refseq_db = decompressed_joint_fasta_file

# # start time Building a new DB, current time: 03/16/2023 16:13:05
# # end time ???

# # (base) 2023-03-17T21:11:40 jovyan@76eefed4ebe2:~/.../blastdbs/refseq
# # $ time gzip -dc refseq.fna.gz | makeblastdb -in - -dbtype nucl -parse_seqids -out refseq.fna.gz -title "Refseq 2023-03-17" -taxid_map refseq.fna.gz.taxon_map.txt


# # Building a new DB, current time: 03/17/2023 21:11:44
# # New DB name:   /home/jovyan/work/blastdbs/refseq/refseq.fna.gz
# # New DB title:  Refseq 2023-03-17
# # Sequence type: Nucleotide
# # Deleted existing Nucleotide BLAST database named /home/jovyan/work/blastdbs/refseq/refseq.fna.gz
# # Keep MBits: T
# # Maximum file size: 3000000000B
# # Adding sequences from FASTA; added 42462451 sequences in 42543.2 seconds.



# # real    717m46.778s
# # user    761m57.365s
# # sys     51m2.045s


# # p = pipeline(`gzip -dc $(joint_fasta_file)`, `makeblastdb -in - -dbtype nucl -parse_seqids -out $(joint_fasta_file) -title $(db_title) -taxid_map $(taxon_map_file)`)
# # @time run(p)
# # @time run(`makeblastdb -in $(decompressed_joint_fasta_file) -dbtype nucl -parse_seqids -out $(decompressed_joint_fasta_file) -title $(db_title) -taxid_map $(taxon_map_file)`)


# # gzip -dc refseq.fna.gz | makeblastdb -in - -dbtype nucl -parse_seqids -out refseq.fna.gz -title "Refseq 2023-03-17" -taxid_map refseq.fna.gz.taxon_map.txt

# # ll -tr refseq*

In [ ]:
# windowmasker and dustmasker - not currently using

# window_masker_step_1_outfile = "$(joint_fasta_file).window_masker.1"
# @time run(`windowmasker -mk_counts -infmt fasta -in $(joint_fasta_file) -out $(window_masker_step_1_outfile)`)
# window_masker_step_2_outfile = "$(joint_fasta_file).window_masker.2"
# @time run(`windowmasker -ustat $(window_masker_step_1_outfile) -dust true -in $(joint_fasta_file) -infmt fasta -out $(window_masker_step_2_outfile)`)

# run(`makeblastdb -in nt –input_type blastdb -dbtype nucl -parse_seqids -mask_data nt.wm.2 -out nt_masked_deduped -title "nt masked and deduped"`)

    # windowmasker -mk_counts -checkdup true -infmt blastdb -in nt -out nt.wm.1
    # windowmasker -ustat nt.wm.1 -dust true -in nt -infmt blastdb -out nt.wm.2
    # makeblastdb -in nt –input_type blastdb -dbtype nucl -parse_seqids -mask_data nt.wm.2 -out nt_masked_deduped -title "nt masked and deduped"

In [ ]:
blastdbs_dir = "$(homedir())/workspace/blastdbs"

# # most generous
# blast_db = "ref_viruses_rep_genomes"
# blastn_task = "blastn"

# less generous
blast_db = "ref_viruses_rep_genomes"
blastn_task = "dc-megablast"

# # validation
# blast_db = "nt_viruses"
# blastn_task = "dc-megablast"

blastdb_dir = mkpath(joinpath(blastdbs_dir, blast_db))
if !isempty(readdir(blastdb_dir))
    @info "blast db detected, using existing"
else
    Mycelia.download_blast_db(db=blast_db, outdir=blastdb_dir, source="ncbi")
end

taxdump_dir = mkpath(joinpath(blastdbs_dir, "taxdump"))
taxdump_tar = joinpath(taxdump_dir, "taxdump.tar.gz")
if !isfile(taxdump_tar)
    run(`wget --quiet https://ftp.ncbi.nih.gov/pub/taxonomy/taxdump.tar.gz --directory-prefix=$(taxdump_dir)`)
end
if isempty(filter(x -> occursin(r"\.dmp$", x), readdir(taxdump_dir)))
    run(`tar -xvzf $(taxdump_tar) --directory $(taxdump_dir)`)
end

In [ ]:
db_path = joinpath(blastdb_dir, blast_db)

In [ ]:
SRR_paths = filter(x -> !occursin(".ipynb_checkpoints", x), readdir(joinpath(data_dir, "SRA"), join=true))
SRR_paths = filter(x -> isfile(joinpath(x, "megahit", "final.contigs.fastg.gfa.fna")), SRR_paths)

In [ ]:
for SRR_path in SRR_paths
    SRR = basename(SRR_path)
    # @show SRR
    contig_info_tsv = "$(SRR_path)/$(SRR).final.contigs.fastg.gfa.fna.blastn.$(blast_db).$(blastn_task).contig_info.tsv"
    if isfile(contig_info_tsv)
        # @info "$(contig_info_tsv) already present, skipping..."
        continue
    end

    assembled_fasta = joinpath(SRR_path, "megahit", "final.contigs.fastg.gfa.fna")

    # # run megablast to get first quick pass at species-level identifications
    # # 12 seconds
    # @time megablast_report = Mycelia.run_blastn(out_dir = SRR_path, fasta = assembled_fasta, blast_db = db_path, task = "megablast")
    # # 53 seconds
    # @time dc_megablast_report = Mycelia.run_blastn(out_dir = SRR_path, fasta = assembled_fasta, blast_db = db_path, task = "dc-megablast")
    # 98 seconds
    # 585.139944 seconds
    @time blastn_report = Mycelia.run_blastn(out_dir = SRR_path, fasta = assembled_fasta, blast_db = db_path, task = blastn_task)

    # 7747 contigs total
    # # 286 hits against 210 contigs
    # megablast_results = Mycelia.parse_blast_report(megablast_report)
    # @show DataFrames.nrow(megablast_results)
    # @show length(unique(megablast_results[!, "query id"]))

    # # 1370 hits across 625 contigs
    # dc_megablast_results = Mycelia.parse_blast_report(dc_megablast_report)
    # @show DataFrames.nrow(dc_megablast_results)
    # @show length(unique(dc_megablast_results[!, "query id"]))

    # # 2432 hits across 900 contigs
    @info "reading in blast report"
    blastn_results = Mycelia.parse_blast_report(blastn_report)
    # @show DataFrames.nrow(blastn_results)
    # @show length(unique(blastn_results[!, "query id"]))

    # viral_contigs = Set(unique(blastn_results[!, "query id"]));
    # # take everything that is a sensitive blastn hit against the viruses db, and then blast those against NT to verify there aren't better non-viral hits
    # viral_fasta = replace(assembled_fasta, ".fna" => ".viral.fna")
    # open(viral_fasta, "w") do io
    #     fastx_io = FASTX.FASTA.Writer(io)
    #     for record in FASTX.FASTA.Reader(open(assembled_fasta))
    #         if (FASTX.identifier(record) in viral_contigs)
    #             write(fastx_io, record)
    #         end
    #     end
    #     close(fastx_io)
    # end
    # @assert length(collect(Mycelia.open_fastx(viral_fasta))) == length(viral_contigs)

    # nt_blastdb_dir = "$(homedir())/workspace/blastdb"
    # nt_blast_db = "nt"
    # nt_blastdb_path = joinpath(nt_blastdb_dir, nt_blast_db)
    # if isdir(blastdb_dir) && !isempty(readdir(blastdb_dir))
    #     @info "blast db detected, using existing"
    # else
    #     Mycelia.download_blast_db(db=nt_blast_db, outdir=nt_blastdb_dir, source="ncbi")
    # end

    # @time blastn_nt_report = Mycelia.run_blastn(out_dir = SRR_path, fasta = viral_fasta, blast_db = nt_blastdb_path, task = "blastn")
    # errored out...

    # this actually had more viral hits than blastn against representative viruses, but only by a bit more
    # nt_megablast_report = joinpath(data_dir, "SRA/SRR6399459/blastn/final.contigs.fastg.gfa.fna.blastn.nt.megablast.txt")

    # intersect(hits, parse.(Int, blastn_results[!, "query id"]))

    # # 18741252 hits (too many!!) across 1142 contigs (not enough?!)
    # # only 17 contigs came back with viral classifications!!
    # nt_megablast_results = Mycelia.parse_blast_report(nt_megablast_report)
    # # @show DataFrames.nrow(nt_megablast_results)
    # # @show length(unique(nt_megablast_results[!, "query id"]))

    # create filtered list of contigs not identified by first pass
    # run dc-megablast to get second quick pass at genus-level identifications

    # read fasta file to determine list of contigs
    # fastx_identifiers = [FASTX.identifier(record) for record in FASTX.FASTA.Reader(open(assembled_fasta))]
    # megablast_hits = unique(map(x -> first(split(x, '\t')), Iterators.filter(l -> !occursin(r"^#", l), eachline(megablast_report))))
    # not_yet_annotated = setdiff(fastx_identifiers, megablast_hits)
    # not_yet_annotated_fasta = replace(assembled_fasta, ".fna" => ".not_yet_annotated.fna")
    # open(not_yet_annotated_fasta, "w") do io
    #     fastx_io = FASTX.FASTA.Writer(io)
    #     for record in FASTX.FASTA.Reader(open(assembled_fasta))
    #         if (FASTX.identifier(record) in not_yet_annotated)
    #             write(fastx_io, record)
    #         end
    #     end
    #     close(fastx_io)
    # end     
    # megablast_results = Mycelia.parse_blast_report(megablast_report)

    # Mycelia.run_blastn(out_dir = SRR_path, fasta = assembled_fasta, blast_db = db_path, task = "dc-megablast")

    # create filered list of contigs not identified by first or second pass

    # Mycelia.run_blastn(out_dir = SRR_path, fasta = assembled_fasta, blast_db = db_path, task = "blastn")

    # @time blast_report = Mycelia.run_blast(out_dir = SRR_path, fasta = assembled_fasta, blast_db = db_path, blast_command = "blastn")

    detected_tax_id_file = "$(assembled_fasta).detected_tax_ids.txt"
    open(detected_tax_id_file, "w") do io
        for taxid in unique(filter(!ismissing, blastn_results[!, "subject tax id"]))
            println(io, taxid)
        end
    end
    taxid_to_lineage_map = Dict(parse(Int, split_line[1]) => split_line[2] for split_line in split.(readlines(`taxonkit lineage --data-dir $(taxdump_dir) $(detected_tax_id_file)`), '\t'))

    qualimap_report_txt = joinpath(SRR_path, "megahit", "qualimap", "genome_results.txt")
    qualimap_contig_coverage_table = Mycelia.parse_qualimap_contig_coverage(qualimap_report_txt)

    blastn_results[!, "lineage"] = map(x -> get(taxid_to_lineage_map, x, ""), blastn_results[!, "subject tax id"])
    blastn_results[!, "% of subject length"] = round.(blastn_results[!, "query length"] ./ blastn_results[!, "subject length"] * 100, digits=3)
    contig_info_table = DataFrames.leftjoin(qualimap_contig_coverage_table, blastn_results, on="Contig" => "query id")

    # re-order columns based on utility
    reordered_columns = [
        "Contig",
        "Length",
        "Mapped bases",
        "Mean coverage",
        "Standard Deviation",
        "% Mapped bases",
        "subject id",
        "subject acc.",
        "subject title",
        "subject tax id",
        "lineage",
        "% identity",
        "% of subject length",
        "evalue",
        "bit score",
        "query length",
        "subject length",
        "alignment length",
        "q. start",
        "q. end",
        "s. start",
        "s. end",
        "identical",
        "mismatches"
    ]
    contig_info_table = contig_info_table[!, reordered_columns]
    sort!(contig_info_table, ["% Mapped bases", "bit score"], rev=true)
    # consider gzipping for large files!
    uCSV.write(contig_info_tsv, coalesce.(contig_info_table, ""), delim='\t')
end

# use 3 way triangulation - reads classify to viral
# contigs classify to viral
# reads map to contigs